In [8]:
# Import required modules
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from wordcloud import WordCloud
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.special import rel_entr

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Use ggplot
matplotlib.style.use('ggplot')

features = ['target_material','target_thickness','pulse_width','energy','spot_size','intensity','power','cutoff_energy']
numeric_features = ['target_thickness', 'pulse_width', 'energy', 'spot_size', 'intensity', 'power', 'cutoff_energy']

# Load original dataset that was used to generate samples
df_original = pd.read_csv('../1_sample_preparation/source/d_clean_remove_small_samples.csv')
# Make sure required features are numeric
df_original[numeric_features] = df_original[numeric_features].astype(float)
df_original.reset_index(drop=True,inplace=True)

# Load synthetic data set
df_synthetic = pd.read_csv('../4_response_extraction/synthetic_data_rows.csv')
# Drop column
df_synthetic.drop(columns=['Unnamed: 0'],inplace=True)
# Make sure required features are numeric
df_synthetic[numeric_features] = df_synthetic[numeric_features].astype(float)
df_synthetic.reset_index(drop=True,inplace=True)

# Create a copy of the original DataFrame
df_synth_raw = df_synthetic.copy()

# Function to remove outliers for a specific feature
def remove_outliers(df, feature, lower_percentile, upper_percentile):
    Q1 = df[feature].quantile(lower_percentile)
    Q3 = df[feature].quantile(upper_percentile)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]

# Remove outliers for all numeric features
for nf in numeric_features:
    df_synth_raw = remove_outliers(df_synth_raw, nf, 0.005, 0.995)

print(f"Original DataFrame length: {len(df_synthetic)}")
print(f"DataFrame length after removing outliers 'RAW' for Inf values: {len(df_synth_raw)} - Rows lost: {len(df_synthetic) - len(df_synth_raw)}")

print("\ndf_original head:",df_original.head())
print("\ndf_original info:",df_original.info())
print("\ndf_original unique prompt_method: NA")
print("\ndf_original unique prompt_short: NA")
print("\ndf_original unique sample_size: NA")
print("\ndf_original unique target_material:",df_original['target_material'].unique())
print("\ndf_original unique model: NA")

print("\ndf_synth_raw head:",df_synth_raw.head())
print("\ndf_synth_raw info:",df_synth_raw.info())
print("\ndf_synth_raw unique prompt_method:",df_synth_raw['prompt_method'].unique())
print("\ndf_synth_raw unique prompt_short:",df_synth_raw['prompt_short'].unique())
print("\ndf_synth_raw unique sample_size:",df_synth_raw['sample_size'].unique())
print("\ndf_synth_raw unique target_material:",df_synth_raw['target_material'].unique())
print("\ndf_synth_raw unique model:",df_synth_raw['model'].unique())

df_synthetic.head()


Original DataFrame length: 163300
DataFrame length after removing outliers 'RAW' for Inf values: 159860 - Rows lost: 3440

df_original head:   target_material  target_thickness  pulse_width  energy  spot_size  \
0         plastic             0.537         30.0   2.427        3.3   
1         plastic             0.293         30.0   2.395        3.3   
2         plastic             0.610         30.0   2.425        3.3   
3         plastic             0.509         30.0   2.344        3.3   
4         plastic             0.527         30.0   2.351        3.3   

      intensity         power  cutoff_energy  
0  6.561000e+20  8.091000e+13            3.3  
1  6.473000e+20  7.983000e+13            3.4  
2  6.554000e+20  8.083000e+13            3.4  
3  6.335000e+20  7.813000e+13            3.4  
4  6.356000e+20  7.838000e+13            3.4  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype

,model,prompt_method,prompt_short,sample_size,target_material,target_thickness,pulse_width,energy,spot_size,intensity,power,cutoff_energy
0,claude-3-5-sonnet-20240620,chain_of_thought,cot,rs_size_5,plastic,0.301,35.0,1.87,3.3,5.120000e+20,5.340000e+13,4.2
1,claude-3-5-sonnet-20240620,chain_of_thought,cot,rs_size_5,gold,0.685,180.0,3.25,3.3,7.210000e+20,1.810000e+13,6.8
2,claude-3-5-sonnet-20240620,chain_of_thought,cot,rs_size_5,aluminium,0.952,42.0,2.39,3.3,6.380000e+20,5.690000e+13,5.1
3,claude-3-5-sonnet-20240620,chain_of_thought,cot,rs_size_5,polypropylene,1.230,320.0,15.60,4.1,1.450000e+21,4.880000e+13,18.3
4,claude-3-5-sonnet-20240620,chain_of_thought,cot,rs_size_5,plastic,0.488,30.0,2.33,3.3,6.300000e+20,7.770000e+13,4.5


In [ ]:
df_original.head()

In [ ]:
# Count the occurrences of each 'target_material'
material_counts = df_synth_raw['target_material'].value_counts()

print('Unique Materials:',len(df_synth_raw['target_material'].unique()))

# Create a WordCloud object
wordcloud = WordCloud(width = 800, height = 400, background_color ='white')

# Generate a word cloud using the frequencies
wordcloud.generate_from_frequencies(material_counts.to_dict())

# Display the WordCloud image:
plt.figure(figsize=(25, 15))
plt.title('Wordcloud of frequent target_material (all models)',y=1.05)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Turn off axis labels

plt.tight_layout()

plt.savefig('./images/word_cloud_target_material.jpg',dpi=800)
plt.show()

In [ ]:
# Count the occurrences of each 'target_material'
material_counts = df_synth_raw['target_material'].value_counts()

# Assuming 'material_counts' holds the value counts of 'target_material' from your DataFrame
top_25_materials = material_counts.head(50)  # Get the top 50 materials for better illustration

# Create a DataFrame from the top 25 materials for easier plotting
top_25_df = pd.DataFrame(top_25_materials).reset_index()
top_25_df.columns = ['target_material', 'count']

# Calculate cumulative percentage
total_rows = df_synthetic.shape[0]  # Total number of rows in the original DataFrame
top_25_df['cumulative_percent'] = top_25_df['count'].cumsum() / total_rows * 100

# Initialize the matplotlib figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(18, 10), sharey=True)  # Share y-axis across subplots

# Left subplot for frequency
sns.barplot(x='count', y='target_material', data=top_25_df, ax=axes[0], hue='target_material',palette='viridis')
axes[0].set_xscale('log')
axes[0].set_xlabel('Frequency (Log Scale)')
axes[0].set_ylabel('Material')
axes[0].set_title('Top 50 Frequent Materials')

# Right subplot for cumulative percentage
sns.lineplot(x='cumulative_percent', y='target_material', data=top_25_df, ax=axes[1],color='black')
axes[1].set_xlabel('Cumulative Percentage (%)')
axes[1].set_title('Total Share of Rows by Material')

# Set x-axis to have a tick every 5 percent
axes[1].xaxis.set_major_locator(ticker.MultipleLocator(5))

# Optionally, you can format the ticks to show them as percentages
axes[1].xaxis.set_major_formatter(ticker.PercentFormatter(xmax=100))

# If you want to limit the x-axis to show up to 100% only
axes[1].set_xlim(40, 100)

# Annotate each point on the line plot
for index, row in top_25_df.iterrows():
    axes[1].annotate(f"{row['cumulative_percent']:.1f}%", 
                     (row['cumulative_percent'], row['target_material']), 
                     textcoords="offset points", 
                     xytext=(-14,-10), 
                     ha='center', 
                     color='black', 
                     size=7,
    )
# Plotting the points as black dots
axes[1].scatter(top_25_df['cumulative_percent'], top_25_df['target_material'], color='black', s=20)  # s is the size of the dot

plt.show()

# Improve layout
plt.tight_layout()

# Save the figure
plt.savefig('./images/top_50_materials_comparison.jpg', dpi=300)
# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

num_models = len(df_synth_raw['model'].unique())
num_features = len(numeric_features)

# Calculate the optimal figure size
fig_width = min(20, 2.5 * num_features)  # Cap the width at 20 inches
fig_height = min(30, 2 * num_models)  # Cap the height at 30 inches

# Initialize the matplotlib figure with subplots
fig, axes = plt.subplots(num_models, num_features, figsize=(fig_width, fig_height))

# If there's only one model, wrap axes in a list to make it 2D
if num_models == 1:
    axes = np.array([axes])

for i, model_name in enumerate(df_synth_raw['model'].unique()):
    # Filter the DataFrame for the specific model
    model_df = df_synth_raw[df_synth_raw['model'] == model_name].copy()

    for j, feature in enumerate(numeric_features):
        # Create a boxplot for the numeric features of this specific model
        sns.boxplot(data=model_df, y=feature, ax=axes[i, j], orient='v', color='red', fliersize=2)
        
        # Set the xlabel on each subplot
        axes[i, j].set_xlabel('')
        
        # Only set the feature name for the bottom row
        if i == num_models - 1:
            axes[i, j].set_xlabel(feature, fontsize=10, fontweight='normal')
        
        # Remove y-label for all subplots
        axes[i, j].set_ylabel('')

        # Rotate x-axis labels for better readability
        axes[i, j].tick_params(axis='x', rotation=45, labelsize=8)
        axes[i, j].tick_params(axis='y', labelsize=8)

        if i % 2 != 0:
            axes[i, j].patch.set_facecolor((0.95, 0.95, 0.95))  # Light grey color
        else:
            axes[i, j].patch.set_facecolor((0.99, 0.99, 0.99))  # Light grey color

    # Add model name to the top left corner of each row
    axes[i, 0].text(-0.3, 1.125, model_name, transform=axes[i, 0].transAxes, 
                    fontsize=12, fontweight='light', ha='left', va='bottom')

    # Extend the shaded background to include the model name area for shaded rows
    if i % 2 == 0:
        axes[i, 0].add_patch(plt.Rectangle((-0.35, -0.1), 10, 1.4, 
                                           fill=True, transform=axes[i, 0].transAxes, 
                                           facecolor=(0.96, 0.96, 0.96), clip_on=False, zorder=0))

# Adjust the layout and spacing
plt.tight_layout()
plt.subplots_adjust(hspace=0.4, wspace=0.435, top=0.935, left=0.15)

# Add a main title
fig.suptitle('Feature Distribution Across Models - Raw', fontsize=16, fontweight='bold', y=0.975)
fig.text(s='Raw data which might include extreme outliers due to mistakes large language models can introduce.',y=0.957,x=0.255,fontsize=12, fontweight='light')

# Save the figure with high resolution
plt.savefig('./images/boxplot_outliers_models_raw.png', dpi=300, bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

def create_kde_plot(data, title, subtitle):
    fig, axes = plt.subplots(2, 4, figsize=(18, 10))
    axes = axes.flatten()

    synthetic_models = [model for model in data['model'].unique() if model != 'original data']
    color_palette = sns.color_palette("husl", n_colors=len(synthetic_models))

    for i, feature in enumerate(numeric_features):
        ax = axes[i]
        
        # Determine the x-axis range
        x_min = max(0, data[feature].quantile(0.0025))  # Use 1st percentile, but never go below 0
        x_max = data[feature].quantile(0.9975)  # Use 99th percentile to exclude extreme outliers
        x_range = x_max - x_min
        x_padding = x_range * 0.1  # Add 10% padding on each side
        
        original_data = data[data['model'] == 'original data']
        if not original_data.empty:
            sns.kdeplot(data=original_data, x=feature, ax=ax, color='black', alpha=1, label='Original Data', bw_adjust=1.5)
        
        for j, model in enumerate(synthetic_models):
            model_data = data[data['model'] == model]

            if model_data[feature].nunique() == 1:
                print(f"  Skipping plot for {model} - all values are the same")
                continue
            
            sns.kdeplot(data=model_data, x=feature, ax=ax, color=color_palette[j], alpha=0.7, label=model, bw_adjust=1.5)
        
        ax.set_title(f"log({feature})", fontsize=14, fontweight='bold')
        ax.set_xlabel('')
        ax.set_ylabel('Density', fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.set_xlim(x_min - x_padding, x_max + x_padding)
        ax.legend().remove()
        
        # Remove scientific notation from x-axis
        ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.2f'))

    handles, labels = axes[0].get_legend_handles_labels()
    axes[-1].legend(handles, labels, title='Models', title_fontsize='15', fontsize='13', loc='center', bbox_to_anchor=(0.5, 0.5), bbox_transform=axes[-1].transAxes)
    axes[-1].axis('off')

    plt.tight_layout()
    fig.suptitle(title, fontsize=20, fontweight='bold', y=1.02)
    fig.text(0.5, 0.97, subtitle, ha='center', fontsize=14, fontstyle='italic')
    plt.subplots_adjust(top=0.9, hspace=0.3, wspace=0.3)

    return fig

df_original_log = df_original.copy()
df_synth_raw_log = df_synth_raw.copy()

# Add 'model' column to df_original and select only numeric features
df_original_log['model'] = 'original data'
df_original_log = df_original_log[['model'] + numeric_features]

# Log transform numeric features
for feature in numeric_features:
    df_original_log[feature]            = np.log1p(df_original_log[feature])
    df_synth_raw_log[feature] = np.log1p(df_synth_raw[feature])

# Add original data to cleaned dataframes
df_clean_raw_with_original = pd.concat([df_synth_raw_log, df_original_log], ignore_index=True)

# Create and save plots for raw data (without inf/NaN values), IQR-cleaned data, and IPR-cleaned data
raw_plot = create_kde_plot(df_clean_raw_with_original, 'Kernel Density Estimation of Log-Transformed Features Across Models (Raw Data)', 'Extreme outliers removed (0.5th to 99.5th percentile)')
raw_plot.savefig('./images/kde_features_across_models_raw_log.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Assuming df_remove_outliers and df_original are already loaded

# Selecting numerical features for PCA
features = ['target_thickness', 'pulse_width', 'energy', 'spot_size', 'intensity', 'power', 'cutoff_energy']

# Ensure df_original has all columns from df_remove_outliers
missing_cols = set(df_synth_raw.columns) - set(df_original.columns)
for col in missing_cols:
    df_original[col] = np.nan

# Add 'source' column
df_synth_raw['source'] = 'synthetic'
df_original['source'] = 'original'

# Drop NaN values only from the synthetic data
df_synth_raw_copy = df_synth_raw.copy()
df_synth_raw_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
df_synth_raw_copy.dropna(subset=features, inplace=True)

# Combine the cleaned synthetic DataFrame with the original DataFrame
combined_df = pd.concat([df_synth_raw_copy, df_original], ignore_index=True)

# Now proceed with PCA on the combined DataFrame
X = combined_df[features].copy()  # Create a copy to avoid SettingWithCopyWarning

# Standardizing the features
X_scaled = StandardScaler().fit_transform(X)

# Proceed with PCA
pca = PCA(n_components=2)  # Example: reducing to 2 principal components
principal_components = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(data=principal_components, columns=['Principal Component 1', 'Principal Component 2'])

# Adding the source and model information back to the PCA DataFrame
pca_df = pd.concat([pca_df, combined_df.reset_index(drop=True)[['source', 'model']]], axis=1)

# Handle NaNs in the 'model' column for original data
pca_df['model'].fillna('original', inplace=True)

# Separate the synthetic and original data points
synthetic_points = pca_df[pca_df['source'] == 'synthetic']
original_points = pca_df[pca_df['source'] == 'original']

# Plotting the PCA result
plt.figure(figsize=(20, 15))
plt.xlim([-2,20])

# Plot synthetic points first
sns.scatterplot(data=synthetic_points, x='Principal Component 1', y='Principal Component 2', hue='model', style='model', palette='Set1', alpha=0.5, legend=True)

# Plot original points on top in black
sns.scatterplot(data=original_points, x='Principal Component 1', y='Principal Component 2', color='black')

plt.title('PCA of Features')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def create_scatter_pca(synthetic, original,synthetic_name):
    # Selecting numerical features for PCA
    features = ['target_thickness', 'pulse_width', 'energy', 'spot_size', 'intensity', 'power', 'cutoff_energy']

    # Ensure original has all columns from synthetic
    missing_cols = set(synthetic.columns) - set(original.columns)
    for col in missing_cols:
        original[col] = np.nan

    # Add 'source' column
    synthetic['source'] = 'synthetic'
    original['source'] = 'original'

    # Drop NaN values only from the synthetic data
    df_synth_raw_copy = synthetic.copy()
    df_synth_raw_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_synth_raw_copy.dropna(subset=features, inplace=True)

    # Combine the cleaned synthetic DataFrame with the original DataFrame
    combined_df = pd.concat([df_synth_raw_copy, original], ignore_index=True)

    # Now proceed with PCA on the combined DataFrame
    X = combined_df[features].copy()

    # Standardizing the features
    X_scaled = StandardScaler().fit_transform(X)

    # Proceed with PCA
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(X_scaled)

    # Create a DataFrame with the principal components
    pca_df = pd.DataFrame(data=principal_components, columns=['Principal Component 1', 'Principal Component 2'])

    # Adding the source and model information back to the PCA DataFrame
    pca_df = pd.concat([pca_df, combined_df.reset_index(drop=True)[['source', 'model']]], axis=1)

    # Handle NaNs in the 'model' column for original data
    pca_df['model'].fillna('original', inplace=True)

    # Separate the synthetic and original data points
    synthetic_points = pca_df[pca_df['source'] == 'synthetic']
    original_points = pca_df[pca_df['source'] == 'original']

    # Get unique models
    unique_models = combined_df['model'].unique()
    unique_models = [model for model in unique_models if isinstance(model, str)]

    # Calculate number of rows and columns for subplots
    n_models = len(unique_models)
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols

    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    axes = axes.flatten()  # Flatten the 2D array of axes to iterate easily

    for i, model_name in enumerate(unique_models):
        ax = axes[i]
        # Plot synthetic points first
        sns.scatterplot(data=synthetic_points[synthetic_points['model'] == model_name], 
                        x='Principal Component 1', y='Principal Component 2', 
                        hue='model', style='model', palette='Set1', alpha=0.5, 
                        legend=False, ax=ax)

        # Plot original points on top in black
        sns.scatterplot(data=original_points, x='Principal Component 1', y='Principal Component 2', 
                        color='black', alpha=0.5, ax=ax)

        ax.set_title(f'PCA - {model_name}')
        ax.set_xlabel('Principal Component 1')
        ax.set_ylabel('Principal Component 2')

    # Remove any unused subplots
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout
    plt.tight_layout()

    # Add a main title
    fig.suptitle(f'PCA of Features - Original Dataset vs Synthetic Data ({synthetic_name.upper()})', 
                 fontsize=16, y=1.02)

    # Save the figure
    plt.savefig(f'./images/pca_all_models_orig_vs_synth_{synthetic_name}.jpg', dpi=300, bbox_inches='tight')
    plt.show()

# Call the function
create_scatter_pca(df_synth_raw, df_original,'raw')

In [ ]:
# Assuming df_remove_outliers and df_original are already loaded

# Selecting numerical features for KL Divergence
features = ['target_thickness', 'pulse_width', 'energy', 'spot_size', 'intensity', 'power', 'cutoff_energy']

# Ensure df_original has all columns from df_remove_outliers
missing_cols = set(df_synth_raw.columns) - set(df_original.columns)
for col in missing_cols:
    df_original[col] = np.nan

# Add 'source' column
df_synth_raw['source'] = 'synthetic'
df_original['source'] = 'original'

# Drop NaN values only from the synthetic data
df_synth_raw_copy = df_synth_raw.copy()
df_synth_raw_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
df_synth_raw_copy.dropna(subset=features, inplace=True)

# Combine the cleaned synthetic DataFrame with the original DataFrame
combined_df = pd.concat([df_synth_raw_copy, df_original], ignore_index=True)

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(combined_df[features])

# Create a DataFrame with the scaled features
scaled_df = pd.DataFrame(scaled_features, columns=features)

# Add the source and model columns back to the scaled DataFrame
scaled_df['source'] = combined_df['source'].values
scaled_df['model'] = combined_df['model'].values

# Function to calculate KL Divergence with smoothing
def calculate_kl_divergence(p, q, epsilon=1e-10):
    p = p + epsilon
    q = q + epsilon
    p = p / np.sum(p)
    q = q / np.sum(q)
    return np.sum(rel_entr(p, q))

# Calculate KL Divergence for each feature within each model
kl_results = {}
models = df_synth_raw['model'].unique()

for model_name in models:
    kl_results[model_name] = {}
    for feature in features:
        # Extract data for the current model
        synthetic_data = scaled_df[(scaled_df['model'] == model_name) & (scaled_df['source'] == 'synthetic')][feature]
        original_data = scaled_df[scaled_df['source'] == 'original'][feature]
        
        # Calculate histograms (distributions) with smoothing
        bins = 30
        hist_synthetic, _ = np.histogram(synthetic_data, bins=bins, density=True)
        hist_original, _ = np.histogram(original_data, bins=bins, density=True)

        # Calculate KL Divergence
        kl_divergence = calculate_kl_divergence(hist_synthetic, hist_original)
        kl_results[model_name][feature] = kl_divergence

# Convert the KL results into a DataFrame
kl_df = pd.DataFrame(kl_results).T
kl_df.reset_index(inplace=True)
kl_df.rename(columns={'index': 'model'}, inplace=True)

# Create a MultiIndex for the columns
kl_df.columns = pd.MultiIndex.from_tuples([('Model', '')] + [('KL Divergence', feature) for feature in kl_df.columns[1:]])

# Print KL Divergence DataFrame
#kl_df

# Calculate the mean of the KL divergence features for each row
kl_df['median_kl'] = kl_df.iloc[:, 2:].median(axis=1)
kl_df['mean_kl'] = kl_df.iloc[:, 2:].mean(axis=1)

# Sort the DataFrame by this new column in descending order
kl_df_sorted = kl_df.sort_values(by='median_kl', ascending=True)
kl_df_sorted.dropna(inplace=True)

# Drop the temporary mean column
# kl_df_sorted = kl_df_sorted.drop(columns=['median_kl'])

# Print the sorted DataFrame
kl_df_sorted

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.special import rel_entr
from scipy.stats import entropy
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import wasserstein_distance

def calculate_wasserstein_distance(x, y):
    if len(x) == 0 or len(y) == 0:
        return np.nan
    
    x = np.asarray(x)
    y = np.asarray(y)
    
    # Calculate Wasserstein distance for each dimension
    distances = [wasserstein_distance(x[:, i], y[:, i]) for i in range(x.shape[1])]
    
    # Return the average distance across all dimensions
    return np.mean(distances)

# Function to calculate KL Divergence with smoothing
def calculate_kl_divergence(p, q, epsilon=1e-10):
    p = p + epsilon
    q = q + epsilon
    p = p / np.sum(p)
    q = q / np.sum(q)
    return np.sum(rel_entr(p, q))

# Function to calculate Maximum Mean Discrepancy (MMD) (memory-efficient version)
def calculate_mmd(x, y, kernel='rbf', batch_size=1000):
    nx, ny = len(x), len(y)
    x = np.asarray(x)
    y = np.asarray(y)
    
    gamma = 1.0 / x.shape[1]
    
    total_kxx = 0
    total_kyy = 0
    total_kxy = 0
    
    for i in range(0, nx, batch_size):
        xi = x[i:i+batch_size]
        kxx = np.mean(np.exp(-gamma * euclidean_distances(xi, xi, squared=True)))
        kxy = np.mean(np.exp(-gamma * euclidean_distances(xi, y, squared=True)))
        total_kxx += kxx * len(xi)
        total_kxy += kxy * len(xi)
    
    for i in range(0, ny, batch_size):
        yi = y[i:i+batch_size]
        kyy = np.mean(np.exp(-gamma * euclidean_distances(yi, yi, squared=True)))
        total_kyy += kyy * len(yi)
    
    total_kxx /= nx
    total_kyy /= ny
    total_kxy /= (nx * ny)
    
    return total_kxx + total_kyy - 2 * total_kxy

# Updated multivariate distance calculation
def calculate_multivariate_distance(synthetic_data, original_data, method):
    if len(synthetic_data) == 0 or len(original_data) == 0:
        return np.nan
    
    synthetic_data = np.asarray(synthetic_data)
    original_data = np.asarray(original_data)
    
    if method == 'kl':
        hist_synthetic, _ = np.histogramdd(synthetic_data, bins=[10]*synthetic_data.shape[1], density=True)
        hist_original, _ = np.histogramdd(original_data, bins=[10]*original_data.shape[1], density=True)
        return calculate_kl_divergence(hist_synthetic.flatten(), hist_original.flatten())
    elif method == 'wasserstein':
        return calculate_wasserstein_distance(synthetic_data, original_data)
    elif method == 'mmd':
        return calculate_mmd(synthetic_data, original_data)
    else:
        raise ValueError(f"Unknown method: {method}")

# Updated main calculation loop
def calculate_distances(scaled_df, grouping_columns, features):
    results = {}
    
    for group, group_data in scaled_df[scaled_df['source'] == 'synthetic'].groupby(grouping_columns):
        synthetic_data = group_data
        original_data = scaled_df[scaled_df['source'] == 'original']
        
        # Count samples
        n_samples = len(synthetic_data)
        
        # Multivariate calculations
        synthetic_multivariate = synthetic_data[features].values
        original_multivariate = original_data[features].values

        print(f"Calculating for group: {group}")
        print(f"Synthetic data shape: {synthetic_multivariate.shape}")
        print(f"Original data shape: {original_multivariate.shape}")
        
        group_results = {
            'n_samples': n_samples,
            'multivariate_kl': calculate_multivariate_distance(synthetic_multivariate, original_multivariate, 'kl'),
            'multivariate_wasserstein': calculate_multivariate_distance(synthetic_multivariate, original_multivariate, 'wasserstein'),
            'multivariate_mmd': calculate_multivariate_distance(synthetic_multivariate, original_multivariate, 'mmd')
        }

        print(f"Results for group {group}: {group_results}")
        
        # Handle different grouping levels
        if isinstance(group, tuple):
            current_level = results
            for level in group[:-1]:
                if level not in current_level:
                    current_level[level] = {}
                current_level = current_level[level]
            current_level[group[-1]] = group_results
        else:
            results[group] = group_results
    
    return results

# Updated function to convert nested dictionary to DataFrame
def nested_dict_to_df(d, index_names):
    rows = []
    def process_dict(current_dict, current_row):
        if isinstance(current_dict, dict):
            if all(isinstance(v, (int, float)) for v in current_dict.values()):
                # This is a leaf node with metric values
                rows.append(current_row + list(current_dict.values()))
            else:
                for key, value in current_dict.items():
                    new_row = current_row + [key]
                    process_dict(value, new_row)
        else:
            # This handles the case where a value might be a single number
            rows.append(current_row + [current_dict])
    
    process_dict(d, [])
    df = pd.DataFrame(rows, columns=index_names + ['n_samples', 'KL Divergence', 'Wasserstein', 'MMD'])
    return df

In [ ]:
# Prepare data
features = ['target_thickness', 'pulse_width', 'energy', 'spot_size', 'intensity', 'power', 'cutoff_energy']
missing_cols = set(df_synth_raw.columns) - set(df_original.columns)
for col in missing_cols:
    df_original[col] = np.nan

df_synth_raw['source'] = 'synthetic'
df_original['source'] = 'original'

df_synth_raw_copy = df_synth_raw.copy()
df_synth_raw_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
df_synth_raw_copy.dropna(subset=features, inplace=True)

combined_df = pd.concat([df_synth_raw_copy, df_original], ignore_index=True)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(combined_df[features])

scaled_df = pd.DataFrame(scaled_features, columns=features)
scaled_df['source'] = combined_df['source'].values
scaled_df['model'] = combined_df['model'].values
scaled_df['prompt_method'] = combined_df['prompt_method'].values
scaled_df['sample_size'] = combined_df['sample_size'].values

# Calculate distances for different groupings
results_model = calculate_distances(scaled_df, ['model'], features)
results_prompt = calculate_distances(scaled_df, ['prompt_method'], features)
results_sample = calculate_distances(scaled_df, ['sample_size'], features)

results_model_prompt = calculate_distances(scaled_df, ['model', 'prompt_method'], features)
results_model_sample_size = calculate_distances(scaled_df, ['model', 'sample_size'], features)

results_model_prompt_sample = calculate_distances(scaled_df, ['model', 'prompt_method', 'sample_size'], features)

# Convert results to DataFrames
df_model = nested_dict_to_df(results_model, ['model'])
df_prompt = nested_dict_to_df(results_prompt, ['prompt_method'])
df_sample = nested_dict_to_df(results_sample, ['sample_size'])

df_model_prompt = nested_dict_to_df(results_model_prompt, ['model', 'prompt_method'])
df_model_sample_size = nested_dict_to_df(results_model_sample_size, ['model', 'sample_size'])

df_model_prompt_sample = nested_dict_to_df(results_model_prompt_sample, ['model', 'prompt_method', 'sample_size'])

# Convert n_samples to integer
for df in [df_model, df_model_prompt, df_model_prompt_sample]:
    df['n_samples'] = df['n_samples'].astype(int)

# Sort DataFrames by KL Divergence
df_model_sorted = df_model.sort_values('KL Divergence')
df_prompt_sorted = df_prompt.sort_values('KL Divergence')
df_sample_size_sorted = df_sample.sort_values('KL Divergence')

df_model_prompt_sorted = df_model_prompt.sort_values('KL Divergence')
df_model_sample_size_sorted = df_model_sample_size.sort_values('KL Divergence')

df_model_prompt_sample_sorted = df_model_prompt_sample.sort_values('KL Divergence')

# Print results
print("Results grouped by model:")
print(df_model_sorted)
print("Results grouped by prompt:")
print(df_prompt_sorted)
print("Results grouped by sample_size:")
print(df_sample_size_sorted)

print("\nResults grouped by model and prompt_method:")
print(df_model_prompt_sorted)
print("\nResults grouped by model and sample_size:")
print(df_model_sample_size_sorted)

print("\nResults grouped by model, prompt_method, and sample_size:")
print(df_model_prompt_sample_sorted)

In [ ]:
df_model_sorted.head(15)

In [ ]:
df_prompt_sorted.head(15)

In [ ]:
df_sample_size_sorted.head(15)

In [ ]:
df_model_prompt_sorted.head(5)

In [ ]:
df_model_sample_size_sorted.head(5)

In [ ]:
df_model_prompt_sample_sorted.head(5)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import griddata

# Assuming df_model_prompt_sorted is already loaded
numeric_metrics = ['KL Divergence', 'Wasserstein', 'MMD']

df_ordered = df_model_prompt_sorted

# Create a figure with subplots for each metric
fig = plt.figure(figsize=(60, 15))

for i, metric in enumerate(numeric_metrics, 1):
    # Order models based on average metric value (reversed)
    model_order = df_ordered.groupby('model')[metric].mean().sort_values(ascending=True).index

    # Order prompt_method based on average metric value (reversed)
    prompt_order = df_ordered.groupby('prompt_method')[metric].mean().sort_values(ascending=True).index

    # Create numeric mappings for ordered models and prompt methods
    model_map = {model: i for i, model in enumerate(reversed(model_order))}
    prompt_map = {method: i for i, method in enumerate(prompt_order)}

    # Create a grid of x, y coordinates
    x = np.array([model_map[m] for m in df_ordered['model']])
    y = np.array([prompt_map[p] for p in df_ordered['prompt_method']])
    z = df_ordered[metric].values

    # Create a grid for the surface plot
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    X, Y = np.meshgrid(xi, yi)

    # Interpolate the z values on the grid
    Z = griddata((x, y), z, (X, Y), method='cubic')

    # Create 3D axes
    ax = fig.add_subplot(1, 3, i, projection='3d')

    # Create a custom colormap (inverted viridis)
    colors = plt.cm.viridis(np.linspace(0, 1, 256))[::-1]
    custom_cmap = LinearSegmentedColormap.from_list("inverted_viridis", colors)

    # Create the surface plot
    surf = ax.plot_surface(X, Y, Z, cmap=custom_cmap, edgecolor='none', alpha=0.8)

    # Set labels and title
    ax.set_zlabel(metric, labelpad=20)
    ax.set_title(f'3D Surface Plot: Model vs Prompt Method vs {metric}', pad=20)

    # Set tick labels
    ax.set_xticks(list(model_map.values()))
    ax.set_xticklabels(list(model_map.keys()), rotation=45, ha='right', va='top')
    ax.set_yticks(list(prompt_map.values()))
    ax.set_yticklabels(list(prompt_map.keys()), rotation=-20, ha='left')

    # Adjust tick label positions
    ax.tick_params(axis='x', which='major', pad=8)
    ax.tick_params(axis='y', which='major', pad=8)

    # Add a color bar
    cbar = fig.colorbar(surf, ax=ax, label=metric, pad=0.1, aspect=30)

    # Set view angle
    ax.view_init(elev=20, azim=-45)  # Adjusted view angle

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [9]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import griddata

numeric_metrics = ['KL Divergence', 'Wasserstein', 'MMD']

index = 0
df_3d = [df_model_sample_size_sorted.dropna(),df_model_prompt_sorted.dropna()][index]
label = ['Sample Size','Prompt Method'][index]
label_groupby = ['sample_size','prompt_method'][index]

# Create a figure with subplots for each metric
fig = plt.figure(figsize=(15, 30))

for i, metric in enumerate(numeric_metrics, 1):
    # Order models based on average metric value (descending order)
    model_order = df_3d.groupby('model')[metric].mean().sort_values(ascending=False).index

    # Order labels based on average metric value (descending order)
    label_order = df_3d.groupby(label_groupby)[metric].mean().sort_values(ascending=True).index

    # Create numeric mappings for ordered models and labels
    model_map = {model: i for i, model in enumerate(model_order)}
    label_map = {size: i for i, size in enumerate(label_order)}

    # Create a grid of x, y coordinates
    x = np.array([model_map[m] for m in df_3d['model']])
    y = np.array([label_map[s] for s in df_3d[label_groupby]])
    z = df_3d[metric].values

    # Create a grid for the surface plot
    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    X, Y = np.meshgrid(xi, yi)

    # Interpolate the z values on the grid
    Z = griddata((x, y), z, (X, Y), method='cubic')

    # Create 3D axes
    ax = fig.add_subplot(3, 1, i, projection='3d')

    # Create a custom colormap (inverted viridis)
    colors = plt.cm.viridis(np.linspace(0, 1, 256))[::-1]
    custom_cmap = LinearSegmentedColormap.from_list("inverted_viridis", colors)

    # Create the surface plot
    surf = ax.plot_surface(X, Y, Z, cmap=custom_cmap, edgecolor='none', alpha=0.8)

    # Set labels and title
    # ax.set_xlabel('Model', labelpad=20)
    # ax.set_ylabel(label, labelpad=20)
    ax.set_zlabel(metric, labelpad=20)
    ax.set_title(f'3D Surface Plot: Model vs {label} vs {metric}', pad=20)

    # Set tick labels
    ax.set_xticks(list(model_map.values()))
    ax.set_xticklabels(list(model_map.keys()), rotation=45, ha='right', va='top')
    ax.set_yticks(list(label_map.values()))
    ax.set_yticklabels(list(label_map.keys()), rotation=-20, ha='left')

    # Adjust tick label positions
    ax.tick_params(axis='x', which='major', pad=8)
    ax.tick_params(axis='y', which='major', pad=8)

    # Set z-axis limits to match the range of metric values
    z_min, z_max = df_3d[metric].min(), df_3d[metric].max()
    ax.set_zlim(z_min, z_max)

    # Add a color bar
    cbar = fig.colorbar(surf, ax=ax, label=metric, pad=0.1, aspect=30)

    # Set view angle
    ax.view_init(elev=20, azim=-45)

    # Print debugging information
    print(f"\nMetric: {metric}")
    print(f"{label} order:", label_order)
    print("Z-axis range:", z_min, z_max)

# Adjust layout
plt.tight_layout()
plt.savefig(f'./images/3dplot_multivar_{label_groupby}')

# Show the plot
plt.show()

NameError: name 'df_model_sample_size_sorted' is not defined

In [10]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from scipy.spatial import cKDTree
from scipy.spatial.distance import pdist, squareform

def calculate_ci_and_effect_size(statistic_func, x, y, alpha=0.05, n_bootstrap=2):
    print('Calculating Boostrap Distributions for CI and Effect Size...')
    combined = np.vstack([x, y])
    n = len(x)
    observed = statistic_func(x, y)
    
    # Bootstrap for confidence interval
    bootstrap_stats = []
    for _ in range(n_bootstrap):
        boot_x = combined[np.random.choice(len(combined), n, replace=True)]
        boot_y = combined[np.random.choice(len(combined), len(y), replace=True)]
        bootstrap_stats.append(statistic_func(boot_x, boot_y))
    
    ci_lower, ci_upper = np.percentile(bootstrap_stats, [alpha/2 * 100, (1 - alpha/2) * 100])
    
    # Effect size (modified Cohen's d)
    bootstrap_std = np.std(bootstrap_stats)
    if bootstrap_std == 0:
        # If standard deviation is zero, use a small constant to avoid division by zero
        effect_size = (observed - np.mean(bootstrap_stats)) / 1e-8
    else:
        effect_size = (observed - np.mean(bootstrap_stats)) / bootstrap_std
    
    return ci_lower, ci_upper, effect_size

# Multivariate Kolmogorov-Smirnov Test
def multivariate_ks_test(x, y, min_samples=5):
    print('Calculating KS Test')

    # Check if either dataset has fewer than min_samples
    if len(x) < min_samples or len(y) < min_samples:
        print(f"Warning: Not enough samples for KL divergence test. x: {len(x)}, y: {len(y)}")
        return np.nan, np.nan, np.nan, np.nan, np.nan

    n, d = x.shape
    m, _ = y.shape
    
    def ks_statistic(x, y):
        z = np.vstack([x, y])
        idx = np.argsort(z, axis=0)
        cdf_x = np.sum(idx < n, axis=1) / n
        cdf_y = np.sum(idx >= n, axis=1) / m
        return np.max(np.abs(cdf_x - cdf_y))
    
    observed = ks_statistic(x, y)
    
    # Approximate p-value using the asymptotic distribution
    n_eff = n * m / (n + m)
    p_value = np.exp(-2 * n_eff * observed**2)
    
    ci_lower, ci_upper, effect_size = calculate_ci_and_effect_size(ks_statistic, x, y)
    
    return observed, p_value, ci_lower, ci_upper, effect_size

# Multivariate KL Divergence Test
def multivariate_kl_divergence_test(x, y, num_permutations=100, k=5, epsilon=1e-10, min_samples=5):
    print('Calculating KL Test')
    
    # Check if either dataset has fewer than min_samples
    if len(x) < min_samples or len(y) < min_samples:
        print(f"Warning: Not enough samples for KL divergence test. x: {len(x)}, y: {len(y)}")
        return np.nan, np.nan, np.nan, np.nan, np.nan

    def kl_divergence(x, y):
        n, m = len(x), len(y)
        d = x.shape[1]
        
        cx = cKDTree(x)
        cy = cKDTree(y)
        
        dx, _ = cx.query(x, k=min(k+1, n), eps=0, p=2)
        dy, _ = cy.query(y, k=min(k+1, m), eps=0, p=2)
        
        dx = dx[:, -1]
        dy = dy[:, -1]
        
        nx = np.array([len(cx.query_ball_point(point, r=dx[i], p=2)) for i, point in enumerate(x)])
        ny = np.array([len(cy.query_ball_point(point, r=dx[i], p=2)) for i, point in enumerate(x)])
        
        # Add small epsilon to avoid log(0)
        nx = np.maximum(nx, epsilon)
        ny = np.maximum(ny, epsilon)
        
        kl = np.mean(np.log(ny) - np.log(nx)) + d * np.log(m / (n - 1))
        return np.maximum(kl, epsilon)  # Return at least epsilon
    
    observed = kl_divergence(x, y)
    combined = np.vstack([x, y])
    n = len(x)
    
    permutation_stats = np.array([
        kl_divergence(combined[np.random.permutation(len(combined))[:n]], 
                      combined[np.random.permutation(len(combined))[n:]])
        for _ in range(num_permutations)
    ])
    
    p_value = np.mean(permutation_stats >= observed)
    ci_lower, ci_upper, effect_size = calculate_ci_and_effect_size(kl_divergence, x, y)
    
    return observed, p_value, ci_lower, ci_upper, effect_size

# Multivariate Anderson-Darling Test
def multivariate_anderson_darling_test(x, y, num_permutations=20, min_samples=5):
    print('Calculating Anderson-Darling Test')

    # Check if either dataset has fewer than min_samples
    if len(x) < min_samples or len(y) < min_samples:
        print(f"Warning: Not enough samples for KL divergence test. x: {len(x)}, y: {len(y)}")
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    def ad_statistic(x, y):
        n, m = len(x), len(y)
        N = n + m
        Z = np.vstack([x, y])
        
        # Compute pairwise distances
        D = squareform(pdist(Z))
        
        # Compute the number of x and y points within each pairwise distance
        ix = np.zeros(N)
        iy = np.zeros(N)
        
        for i in range(N):
            ix[i] = (D[i, :n] <= D[i, i]).sum() - (i < n)
            iy[i] = (D[i, n:] <= D[i, i]).sum() - (i >= n)
        
        # Compute the AD statistic
        ad = (1 / (n * m)) * np.sum((ix * (N - iy) - (n - ix) * iy)**2 / ((ix + iy) * (N - ix - iy) + 1e-8))
        return ad

    observed = ad_statistic(x, y)
    combined = np.vstack([x, y])
    n = len(x)
    
    permutation_stats = np.array([
        ad_statistic(combined[np.random.permutation(len(combined))[:n]], 
                     combined[np.random.permutation(len(combined))[n:]])
        for _ in range(num_permutations)
    ])
    
    p_value = np.mean(permutation_stats >= observed)
    ci_lower, ci_upper, effect_size = calculate_ci_and_effect_size(ad_statistic, x, y)
    
    return observed, p_value, ci_lower, ci_upper, effect_size

# Updated calculate_distances function
def calculate_distances(scaled_df, grouping_columns, features):
    results = {}
    
    for group, group_data in scaled_df[scaled_df['source'] == 'synthetic'].groupby(grouping_columns):
        synthetic_data = group_data
        original_data = scaled_df[scaled_df['source'] == 'original']
        
        n_samples = len(synthetic_data)
        
        synthetic_multivariate = synthetic_data[features].values
        original_multivariate = original_data[features].values

        print(f"Calculating for group: {group}")
        print(f"Synthetic data shape: {synthetic_multivariate.shape}")
        print(f"Original data shape: {original_multivariate.shape}")
        
        # Perform tests
        ks_stat, ks_p, ks_ci_lower, ks_ci_upper, ks_effect = multivariate_ks_test(synthetic_multivariate, original_multivariate)
        kl_stat, kl_p, kl_ci_lower, kl_ci_upper, kl_effect = multivariate_kl_divergence_test(synthetic_multivariate, original_multivariate)
        ad_stat, ad_p, ad_ci_lower, ad_ci_upper, ad_effect = multivariate_anderson_darling_test(synthetic_multivariate, original_multivariate)
        
        group_results = {
            'n_samples': n_samples,
            'ks_statistic': ks_stat,
            'ks_p_value': ks_p,
            'ks_ci_lower': ks_ci_lower,
            'ks_ci_upper': ks_ci_upper,
            'ks_effect_size': ks_effect,
            'kl_statistic': kl_stat,
            'kl_p_value': kl_p,
            'kl_ci_lower': kl_ci_lower,
            'kl_ci_upper': kl_ci_upper,
            'kl_effect_size': kl_effect,
            'ad_statistic': ad_stat,
            'ad_p_value': ad_p,
            'ad_ci_lower': ad_ci_lower,
            'ad_ci_upper': ad_ci_upper,
            'ad_effect_size': ad_effect
        }

        print(f"Results for group {group}: {group_results}")
        
        # Handle different grouping levels (as before)
        if isinstance(group, tuple):
            current_level = results
            for level in group[:-1]:
                if level not in current_level:
                    current_level[level] = {}
                current_level = current_level[level]
            current_level[group[-1]] = group_results
        else:
            results[group] = group_results
    
    return results

# Updated nested_dict_to_df function
def nested_dict_to_df(d, index_names):
    rows = []
    def process_dict(current_dict, current_row):
        if isinstance(current_dict, dict):
            if all(isinstance(v, (int, float)) for v in current_dict.values()):
                rows.append(current_row + list(current_dict.values()))
            else:
                for key, value in current_dict.items():
                    new_row = current_row + [key]
                    process_dict(value, new_row)
        else:
            rows.append(current_row + [current_dict])
    
    process_dict(d, [])
    df = pd.DataFrame(rows, columns=index_names + ['n_samples', 'KS Statistic', 'KS p-value',
                                                   'KL Statistic', 'KL p-value',
                                                   'AD Statistic', 'AD p-value'])
    return df

# Add significance indicators
def add_significance(df, alpha=0.05):
    df['KS Significant'] = df['KS p-value'] < alpha
    df['KL Significant'] = df['KL p-value'] < alpha
    df['AD Significant'] = df['AD p-value'] < alpha
    return df

In [11]:
# Prepare data
features = ['target_thickness', 'pulse_width', 'energy', 'spot_size', 'intensity', 'power', 'cutoff_energy']
missing_cols = set(df_synth_raw.columns) - set(df_original.columns)
for col in missing_cols:
    df_original[col] = np.nan

df_synth_raw['source'] = 'synthetic'
df_original['source'] = 'original'

df_synth_raw_copy = df_synth_raw.copy()
df_synth_raw_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
df_synth_raw_copy.dropna(subset=features, inplace=True)

combined_df = pd.concat([df_synth_raw_copy, df_original], ignore_index=True)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(combined_df[features])

scaled_df = pd.DataFrame(scaled_features, columns=features)
scaled_df['source'] = combined_df['source'].values
scaled_df['model'] = combined_df['model'].values
scaled_df['prompt_method'] = combined_df['prompt_method'].values
scaled_df['sample_size'] = combined_df['sample_size'].values

# Calculate distances for different groupings
results_model = calculate_distances(scaled_df, ['model'], features)
results_prompt = calculate_distances(scaled_df, ['prompt_method'], features)
results_sample = calculate_distances(scaled_df, ['sample_size'], features)

results_model_prompt = calculate_distances(scaled_df, ['model', 'prompt_method'], features)
results_model_sample_size = calculate_distances(scaled_df, ['model', 'sample_size'], features)

results_model_prompt_sample = calculate_distances(scaled_df, ['model', 'prompt_method', 'sample_size'], features)

# Convert results to DataFrames
df_model = nested_dict_to_df(results_model, ['model'])
df_prompt = nested_dict_to_df(results_prompt, ['prompt_method'])
df_sample = nested_dict_to_df(results_sample, ['sample_size'])

df_model_prompt = nested_dict_to_df(results_model_prompt, ['model', 'prompt_method'])
df_model_sample_size = nested_dict_to_df(results_model_sample_size, ['model', 'sample_size'])

df_model_prompt_sample = nested_dict_to_df(results_model_prompt_sample, ['model', 'prompt_method', 'sample_size'])

# Apply significance indicators to all result DataFrames
df_model = add_significance(df_model)
df_prompt = add_significance(df_prompt)
df_sample = add_significance(df_sample)
df_model_prompt = add_significance(df_model_prompt)
df_model_sample_size = add_significance(df_model_sample_size)
df_model_prompt_sample = add_significance(df_model_prompt_sample)

# Convert n_samples to integer
for df in [df_model, df_model_prompt, df_model_prompt_sample]:
    df['n_samples'] = df['n_samples'].astype(int)

# Sort DataFrames by AD Statistic
df_model_sorted = df_model.sort_values('AD Statistic')
df_prompt_sorted = df_prompt.sort_values('AD Statistic')
df_sample_size_sorted = df_sample.sort_values('AD Statistic')

df_model_prompt_sorted = df_model_prompt.sort_values('AD Statistic')
df_model_sample_size_sorted = df_model_sample_size.sort_values('AD Statistic')

df_model_prompt_sample_sorted = df_model_prompt_sample.sort_values('AD Statistic')

# Print results with significance indicators
print("Results grouped by model:")
print(df_model_sorted)
print("Results grouped by prompt:")
print(df_prompt_sorted)
print("Results grouped by sample_size:")
print(df_sample_size_sorted)

print("\nResults grouped by model and prompt_method:")
print(df_model_prompt_sorted)
print("\nResults grouped by model and sample_size:")
print(df_model_sample_size_sorted)

print("\nResults grouped by model, prompt_method, and sample_size:")
print(df_model_prompt_sample_sorted)

Calculating for group: ('claude-3-5-sonnet-20240620',)
Synthetic data shape: (26997, 7)
Original data shape: (1067, 7)
Calculating KS Test
Calculating Boostrap Distributions for CI and Effect Size...
Calculating KL Test
Calculating Boostrap Distributions for CI and Effect Size...
Calculating Anderson-Darling Test
Calculating Boostrap Distributions for CI and Effect Size...
Results for group ('claude-3-5-sonnet-20240620',): {'n_samples': 26997, 'ks_statistic': 0.002663456757439709, 'ks_p_value': 0.9855425011325379, 'ks_ci_lower': 0.002663456757439709, 'ks_ci_upper': 0.002663456757439709, 'ks_effect_size': 0.0, 'kl_statistic': 1e-10, 'kl_p_value': 1.0, 'kl_ci_lower': 1e-10, 'kl_ci_upper': 1e-10, 'kl_effect_size': 0.0, 'ad_statistic': 9.486000782701375, 'ad_p_value': 1.0, 'ad_ci_lower': 33.383384191833116, 'ad_ci_upper': 35.9158116372059, 'ad_effect_size': -18.879448901019362}
Calculating for group: ('claude-3-sonnet-20240229',)
Synthetic data shape: (24200, 7)
Original data shape: (1067,